<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
data = {
    'mktId': 'KSQ',
    'trdDd': today,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901',
}
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_key = requests.post(gen_url, data=data)

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
r = requests.post(down_url, data={'code':gen_key.text})
r.encoding = 'EUC-KR'

In [5]:
from io import StringIO

df = pd.read_csv(StringIO(r.text))

In [6]:
df

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,4010,-105,-2.55,185548767130
1,054620,APS홀딩스,KOSDAQ,금융,16400,-1000,-5.75,334465224400
2,265520,AP시스템,KOSDAQ,반도체,26100,-850,-3.15,377933924700
3,211270,AP위성,KOSDAQ,통신장비,17600,-300,-1.68,265448550400
4,013720,CBI,KOSDAQ,운송장비·부품,3460,-80,-2.26,175414301500
...,...,...,...,...,...,...,...,...
1509,024060,흥구석유,KOSDAQ,유통,8000,150,1.91,120000000000
1510,010240,흥국,KOSDAQ,기계·장비,9440,-110,-1.15,116326250240
1511,189980,흥국에프엔비,KOSDAQ,음식료·담배,3390,80,2.42,130132360530
1512,037440,희림,KOSDAQ,기타서비스,6880,-210,-2.96,95786628000


In [7]:
'''
%%timeit
df.loc[df['시가총액'].between(30000000000,70000000000), '종목명']

%%timeit
df[df['시가총액'].between(30000000000,70000000000)]['종목명']
'''

The slowest run took 33.07 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 561 µs per loop


In [9]:
boundary = df.loc[df['시가총액'].between(30000000000,70000000000)]
boundary

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
10,056000,COWON,KOSDAQ,디지털컨텐츠,2860,0,0.00,56399843500
12,083660,CSA 코스믹,KOSDAQ,화학,1510,-15,-0.98,57280512140
20,050120,ES큐브,KOSDAQ,유통,4630,-70,-1.49,62801718180
22,900290,GRT,KOSDAQ,금융,951,0,0.00,64073625000
35,099520,ITX-AI,KOSDAQ,통신장비,1285,0,0.00,37652662655
...,...,...,...,...,...,...,...,...
1462,079170,한창산업,KOSDAQ,금속,9760,150,1.56,50752000000
1465,002680,한탑,KOSDAQ,음식료·담배,1370,-30,-2.14,38729883560
1474,900270,헝셩그룹,KOSDAQ,금융,501,-7,-1.38,40080000000
1487,011080,형지I&C,KOSDAQ,섬유·의류,1425,-55,-3.72,55511826900


In [10]:
!pip install OpenDartReader

In [11]:
import OpenDartReader

api_key = '4b0512c473e52e1cf59f8ce8380b2b30904f381a'
dart = OpenDartReader(api_key) 

In [24]:
search_start = '2021-08-20'
search_end = '2021-09-28'

In [25]:
dart_df = dart.list(start=search_start, end=search_end, kind='A')
found = list(set(dart_df['stock_code']).intersection(set(boundary['종목코드'])))
out_df = dart_df[dart_df['stock_code'].isin(found)]

In [26]:
out_df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
5,00818472,이큐셀,160600,K,사업보고서 (2021.06),20210917000607,이큐셀,20210917,연
6,00814786,스카이이앤엠,131100,K,사업보고서 (2021.06),20210917000602,스카이이앤엠,20210917,
10,00939942,포시에스,189690,K,[기재정정]사업보고서 (2021.06),20210917000423,포시에스,20210917,연
13,00138747,아세아텍,050860,K,사업보고서 (2021.06),20210917000175,아세아텍,20210917,
18,01101041,패션플랫폼,225590,K,[기재정정]반기보고서 (2021.06),20210917000063,패션플랫폼,20210917,
42,00957568,에스엘바이오닉스,214310,K,[기재정정]반기보고서 (2021.06),20210903000352,에스엘바이오닉스,20210903,
43,00642541,디엔에이링크,127120,K,[기재정정]사업보고서 (2020.12),20210903000235,디엔에이링크,20210903,연
45,00111218,KD,044180,K,[기재정정]반기보고서 (2021.06),20210902000297,KD,20210902,
0,01169434,오가닉티코스메틱,900300,K,반기보고서 (2021.06),20210830000797,오가닉티코스메틱,20210830,
2,01160512,헝셩그룹,900270,K,반기보고서 (2021.06),20210830000763,헝셩그룹,20210830,


In [118]:
def get_recent_finstate(corp_code='', year=2021):
  quarters = ['11011', '11014', '11012', '11013']
  for q in quarters:
    # '11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서
    df = dart.finstate(corp_code, year, reprt_code=q)
    if type(df) != type(None):
      return df[df['fs_nm'] == '연결재무제표']

In [119]:
def get_incomestatement(df):
  is_df = df.loc[df['sj_nm'] == '손익계산서']
  t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount', 'thstrm_add_amount', 'frmtrm_add_amount']]
  t.columns = ['published_date', 'date', 'name', 'current03', 'previous03', 'current12', 'previous12']
  return t

def get_is_score(is_df):
  is_score = {}
  for n in is_df['name']:
    current = str_int(is_df.loc[is_df['name'] == n, 'current03'].values[0])
    previous = str_int(is_df.loc[is_df['name'] == n, 'previous03'].values[0])
    is_score[n] = get_growth_rate(previous, current)
  return is_score

In [120]:
def get_balancesheet(df):
  bs_df = df.loc[df['sj_nm'] == '재무상태표']
  t = bs_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
  t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_bs_score(bs_df):
  bs_score = {}
  for n in bs_df['name']:
    current = str_int(bs_df.loc[bs_df['name'] == n, 'current03'].values[0])
    previous = str_int(bs_df.loc[bs_df['name'] == n, 'previous03'].values[0])
    bs_score[n] = get_growth_rate(previous, current)
  return bs_score

In [121]:
def get_growth_rate(previous, current):
  return (current-previous)/abs(previous)

def str_int(s):
  return int(s.replace(',',''))

In [122]:
for code in out_df['corp_code']:
  try:
    df = get_recent_finstate(code)
    is_df = get_incomestatement(df)
    is_score = get_is_score(is_df)

    bs_df = get_balancesheet(df)
    bs_score = get_bs_score(bs_df)

    print(out_df.loc[out_df['corp_code'] == code, 'rcept_dt'].values[0])
    print(out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0])
    print(is_score)
    print(bs_score)
    print()
  except:
    print('ERROR', out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0])
    print()

ERROR 이큐셀

ERROR 스카이이앤엠

ERROR 포시에스

ERROR 아세아텍

20210917
패션플랫폼
{'매출액': 0.5398933437291242, '영업이익': 27.566627249716422, '법인세차감전 순이익': 90.22381715573147, '당기순이익': 114.76843857828506}
{'유동자산': 0.3405358974998343, '비유동자산': 0.13044835799546836, '자산총계': 0.27688705365982036, '유동부채': 0.16859273903884534, '비유동부채': -0.1680172297121108, '부채총계': 0.1253252036725048, '자본금': 0.0, '이익잉여금': 0.5722563456303145, '자본총계': 0.32471405184614743}

20210903
에스엘바이오닉스
{'매출액': 0.09733411441535623, '영업이익': -18.374621013120063, '법인세차감전 순이익': -0.586948588603823, '당기순이익': -0.5785254647167642}
{'유동자산': 0.6393425372755057, '비유동자산': -0.056903781152389324, '자산총계': 0.13344773806138877, '유동부채': 0.5024524155415482, '비유동부채': -0.2763960009663205, '부채총계': 0.4553923570080081, '자본금': 0.03831579916826594, '이익잉여금': -0.1104762148915406, '자본총계': 0.0111800075442885}

20210903
디엔에이링크
{'매출액': 0.183512389148573, '영업이익': -0.2167522820923746, '법인세차감전 순이익': 0.28456699332782504, '당기순이익': 0.28456699332782504}
{'유동자산': -0.1995904713666909, '비